In [2]:
import awswrangler as wr
import pandas as pd
import datetime

In [ ]:
dfOriginal = wr.s3.read_parquet(path='s3://cchc-dw-qa-analytics/ts_real_presupuesto_gerencias/')

dfOriginal.columns.to_list()

In [ ]:
df = dfOriginal.copy()

df = df[(df.cr_nombre == 'CÁMARAS REGIONALES') & (df.fecha > '2021-01-01')]
#df = df[(df.area == 'CÁMARAS REGIONALES') & (df.fecha != 2023)]

df

In [ ]:
dfBNE = wr.s3.read_parquet(path='s3://cchc-dw-qa-analytics/bne_trabajadores/')

In [ ]:
df = dfBNE.copy()

df.columns.to_list()

8/2 
WORK

In [5]:
dfBNE = wr.s3.read_parquet(path='s3://cchc-dw-dev-raw/bne/oficios_cvs_descargados/')


dfBNE

#antes del filtro sector 5821


,fecha_alta,comuna,region,total,ocupacion
0,2021-03-12,Antofagasta,Antofagasta,3,<NA>
1,2021-03-13,Viña del Mar,Valparaíso,2,<NA>
2,2021-03-16,Viña del Mar,Valparaíso,2,<NA>
3,2021-03-31,Antofagasta,Antofagasta,1,<NA>
4,2021-04-05,Santiago,Metropolitana,9,<NA>
...,...,...,...,...,...
5816,2023-01-03,Quilpué,Valparaíso,1,Maestro(a) obras civiles
5817,2023-01-03,Quilpué,Valparaíso,5,Prevencionista de riesgo
5818,2023-01-03,Talcahuano,Bío Bío,1,Ayudante de faena seca
5819,2023-01-03,Talcahuano,Bío Bío,4,Capataz de obra gruesa y obras preliminares


In [ ]:
df = wr.s3.read_parquet(path='s3://cchc-dw-dev-raw/bne/bne_usuarios_empresas/')

df = df[df.is_activo == True]

df
#151 registros

In [3]:
sii = wr.s3.read_parquet(path='s3://cchc-dw-qa-analytics/sii_socios_insolvencia/', columns=["rut","razon_social","tamano_de_empresa","rubro_economico","subrubro_economico","socio_cchc","estado"])
bolcon = wr.s3.read_parquet(path="s3://cchc-dw-dev-staging/boletin_concursal/insolvencias/")

In [ ]:
dfSii = sii.copy()
dfBolcon = bolcon.copy()

dfSii = dfSii[dfSii.estado == 'ACTIVO']
dfSii.fillna(value={'rut':-1})


dfSii = dfSii.drop_duplicates(subset=['rut'],keep='last')
df = dfBolcon.merge(dfSii,'left','rut')

#listadf=df.rut.drop_duplicates().to_list()
df = df[df.estado == 'ACTIVO']

df
#listaBolcon=dfBolcon.rut.drop_duplicates().to_list()

#df = df[df.rut.isnull()]
#for element in listadf:
#    if element not in listaBolcon:
#        print(element)

#print("bolcon: ",len(bolcon), " sii: ",len(sii), " df: " ,len(df))

In [4]:
len(bolcon)


604847

In [72]:
nuevos_df = wr.s3.read_parquet(f"s3://cchc-dw-dev-raw/_bolcon/insolvencias/year=2022/month=11/day=14/", dataset=True)
nuevos_df["fecha_publicacion"] = pd.to_datetime(nuevos_df.fecha_publicacion, format="%d/%m/%Y")



anterior_df = wr.s3.read_parquet("s3://cchc-dw-dev-staging/boletin_concursal/insolvencias/",)

ruts_anterior = anterior_df[anterior_df.proceso_finalizado.isnull()].rut.fillna("CORRECCION:" + anterior_df.rol).drop_duplicates().to_list()
ruts_nuevos = nuevos_df.rut.fillna("CORRECCION:" + nuevos_df.rol).drop_duplicates().to_list()


#Buscar los ruts en el dataframe antiguos que no existan en el nuevo dataframe para marcarlos como finalizados, o sea, ya no son insolventes
ruts_finalizados = [x for x in ruts_anterior if x not in ruts_nuevos]

anterior_df.loc[ anterior_df.rut.isin(ruts_finalizados), "fecha_ejecucion"] = datetime.date(2022, 11, 7).strftime("%Y-%m-%d")
anterior_df.loc[ anterior_df.rut.isin(ruts_finalizados),  "proceso_finalizado"] = True

ruts_nuevos = [x for x in ruts_nuevos if x not in ruts_anterior]
nuevos_procesos = nuevos_df[nuevos_df.rut.fillna("CORRECCION:" + nuevos_df.rol).isin(ruts_nuevos)].copy()


df = pd.concat([anterior_df, nuevos_procesos])

print("Agregados", len(ruts_nuevos), "registros")
print(ruts_nuevos)
print("Marcados", len(ruts_finalizados), "como finalizados")

sii = wr.s3.read_parquet(path='s3://cchc-dw-qa-analytics/sii_socios_insolvencia/', columns=["rut","razon_social","tamano_de_empresa","rubro_economico","subrubro_economico","socio_cchc","estado"])

sii = sii[(sii.estado == 'ACTIVO') & (sii.rut.isin(ruts_nuevos))]
sii.fillna(value={'rut':-1})
sii = sii.drop_duplicates(subset=['rut'],keep='last')

df = df.merge(sii,'left','rut')

wr.s3.to_parquet(
    df=df,
    path="s3://cchc-dw-dev-staging/boletin_concursal/insolvencias/",
    dataset=True,
    mode="overwrite",
    table="bolcon_insolvencias_historico",
    database="staging_dev",
    schema_evolution=True,
    dtype={
        "fecha_ejecucion" : "string",
        "proceso_finalizado" : "boolean"
    }
)

Agregados 72 registros
['21561866-8', '13310748-7', '17475123-4', '17610139-3', '17009271-6', '76449287-0', '13543878-2', '17930790-1', '18508257-1', '76247325-9', '10315427-8', '16210418-7', '76380036-9', '76214882-K', '15152189-4', '17835536-8', '76070355-9', '16383501-0', '80207900-1', '18389155-3', '16875658-5', '18616900-K', '16225685-8', '16732454-1', '8995704-4', '16827277-4', '16008954-7', '15530837-0', '13516007-5', '12182905-3', '16572113-6', '18909250-4', '13716784-0', '8718942-2', '18416051-K', '8470860-7', '13772221-6', '15547491-2', '14284526-1', '26083549-1', '12880547-8', '16865915-6', '16348498-6', '8750043-8', '25112487-6', '10121003-0', '14260920-7', '14344327-2', '11709640-8', '16058081-K', '18155737-0', '12560878-7', '16010029-K', '16319016-8', '13995266-9', '15411535-8', '76942802-k', '76133054-3', '76771987-6', '15598980-7', '11180044-8', '76404432-0', '12018435-0', '76524928-7', '17973583-0', '83311400-K', '76292422-6', '76741243-6', '19345080-6', '15668419-8', 

{'paths': ['s3://cchc-dw-dev-staging/boletin_concursal/insolvencias/fad0689f5d934a328912bbadfc9b2f5c.snappy.parquet'],
 'partitions_values': {}}